In [4]:
import pandas as pd
#from SPARQLWrapper import SPARQLWrapper, JSON
#import ipython_sparql
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF, CSV, TSV
from bs4 import BeautifulSoup
import requests
import re
import numpy as np

In [14]:
data = pd.read_csv("../data/Suicides_wikipedia_scrapping_fr.csv", encoding="utf-8-sig", delimiter=",")
data.head()

,Name,Link,Content
0,Adolfo_Couve,https://fr.wikipedia.org/wiki/Adolfo_Couve,"['id=""Biographie"">Biographie[modifier', '|', '..."
1,Boaz_Arad,https://fr.wikipedia.org/wiki/Boaz_Arad,"['id=""Biographie"">Biographie[modifier', '|', '..."
2,Ralph_Barton,https://fr.wikipedia.org/wiki/Ralph_Barton,"['id=""Biographie"">Biographie[modifier', '|', '..."
3,Mark_Campos,https://fr.wikipedia.org/wiki/Mark_Campos,"['id=""Biographie"">Biographie[modifier', '|', '..."
4,Michel_Caron_(artiste),https://fr.wikipedia.org/wiki/Michel_Caron_(ar...,"['id=""Biographie"">Biographie[modifier', '|', '..."


In [15]:
listeName = data["Name"].values

## Requests for abstract, birthdate, birthplace, deathdate, deathplace, and gender

In [17]:
abstract = []
birthdate = []
birthplace = []
deathdate = []
deathplace = []
gender = []

In [16]:
i = 0
len(listeName)

2099

In [18]:
for name in range(len(listeName)):
    if(name % 201 == 0):
        print((name/len(listeName)*100),"% du travail effectué !")
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?abstract ?birthdate ?deathdate ?birthplace ?deathplace ?gender
    WHERE { <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:abstract ?abstract.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:birthDate ?birthdate.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathDate ?deathdate.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:birthPlace ?birthplace.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathPlace ?deathplace.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> foaf:gender ?gender}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if (results["results"]["bindings"] != []):
        results = results["results"]["bindings"][0]
        abstract.append(results["abstract"]["value"])
        birthdate.append(results["birthdate"]["value"])
        birthplace.append(results["birthplace"]["value"])
        deathdate.append(results["deathdate"]["value"])
        deathplace.append(results["deathplace"]["value"])
        gender.append(results["gender"]["value"])
    else:
        i+=1
        if (i%10 == 0):
            print("nombre de données manquantes :", i)
        abstract.append("")
        birthdate.append("")
        birthplace.append("")
        deathdate.append("")
        deathplace.append("")
        gender.append("")
print("il manque", (i/len(listeName))*100,"% des données")

0.0 % du travail effectué !
nombre de données manquantes : 10
nombre de données manquantes : 20
nombre de données manquantes : 30
nombre de données manquantes : 40
nombre de données manquantes : 50
nombre de données manquantes : 60
9.575988565983803 % du travail effectué !
nombre de données manquantes : 70
nombre de données manquantes : 80
nombre de données manquantes : 90
nombre de données manquantes : 100
nombre de données manquantes : 110
nombre de données manquantes : 120
nombre de données manquantes : 130
19.151977131967605 % du travail effectué !
nombre de données manquantes : 140
nombre de données manquantes : 150
nombre de données manquantes : 160
nombre de données manquantes : 170
nombre de données manquantes : 180
nombre de données manquantes : 190
nombre de données manquantes : 200
nombre de données manquantes : 210
28.727965697951408 % du travail effectué !
nombre de données manquantes : 220
nombre de données manquantes : 230
nombre de données manquantes : 240
nombre de don

## The birthplace and deathplace without links

In [20]:
birthplace_clean = [""]*len(birthplace)
deathplace_clean = [""]*len(deathplace)
for i in range(len(birthplace)):
    for j in range(28, len(birthplace[i])):
        birthplace_clean[i] += birthplace[i][j]
for i in range(len(deathplace)):
    for j in range(28, len(deathplace[i])):
        deathplace_clean[i] += deathplace[i][j]

In [21]:
data["abstract"] = abstract
data["birthdate"] = birthdate
data["birthplace_clean"] = birthplace_clean
data["birthplace"] = birthplace
data["deathdate"] = deathdate
data["deathplace_clean"] = deathplace_clean
data["deathplace"] = deathplace
data["gender"] = gender

data.head()

,Name,Link,Content,abstract,birthdate,birthplace_clean,birthplace,deathdate,deathplace_clean,deathplace,gender
0,Adolfo_Couve,https://fr.wikipedia.org/wiki/Adolfo_Couve,"['id=""Biographie"">Biographie[modifier', '|', '...","Adolfo Couve Rioseco (Valparaíso, 28 de marzo ...",1940-3-28,Valparaíso,http://dbpedia.org/resource/Valparaíso,1998-3-11,"Cartagena,_Chile","http://dbpedia.org/resource/Cartagena,_Chile",male
1,Boaz_Arad,https://fr.wikipedia.org/wiki/Boaz_Arad,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
2,Ralph_Barton,https://fr.wikipedia.org/wiki/Ralph_Barton,"['id=""Biographie"">Biographie[modifier', '|', '...",Ralph Waldo Emerson Barton (14 août 1891 – 19 ...,1891-8-14,"Kansas_City,_Missouri","http://dbpedia.org/resource/Kansas_City,_Missouri",1931-5-19,"New_York_City,_New_York","http://dbpedia.org/resource/New_York_City,_New...",male
3,Mark_Campos,https://fr.wikipedia.org/wiki/Mark_Campos,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
4,Michel_Caron_(artiste),https://fr.wikipedia.org/wiki/Michel_Caron_(ar...,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,


## Dataframe where the sparql request does not worked (for analysis)

In [22]:
data_errors = data[(data['abstract'] == "")]
#data = data[(data['Content'] != [])]
print("la longueur finale du dataframe d'erreurs est :", data_errors.shape[0])
data_errors.head()

la longueur finale du dataframe d'erreurs est : 981


,Name,Link,Content,abstract,birthdate,birthplace_clean,birthplace,deathdate,deathplace_clean,deathplace,gender
1,Boaz_Arad,https://fr.wikipedia.org/wiki/Boaz_Arad,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
3,Mark_Campos,https://fr.wikipedia.org/wiki/Mark_Campos,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
4,Michel_Caron_(artiste),https://fr.wikipedia.org/wiki/Michel_Caron_(ar...,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
5,Cassandre_(graphiste),https://fr.wikipedia.org/wiki/Cassandre_(graph...,"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,
6,Jack_Cole_(auteur),https://fr.wikipedia.org/wiki/Jack_Cole_(auteur),"['id=""Biographie"">Biographie[modifier', '|', '...",,,,,,,,


## Dataframe where the sparql requests worked

In [23]:
data = data[(data['abstract'] != "")]
#data = data[(data['Content'] != [])]
print("la longueur finale du dataframe est :", data.shape[0])
data.head()

la longueur finale du dataframe est : 1118


,Name,Link,Content,abstract,birthdate,birthplace_clean,birthplace,deathdate,deathplace_clean,deathplace,gender
0,Adolfo_Couve,https://fr.wikipedia.org/wiki/Adolfo_Couve,"['id=""Biographie"">Biographie[modifier', '|', '...","Adolfo Couve Rioseco (Valparaíso, 28 de marzo ...",1940-3-28,Valparaíso,http://dbpedia.org/resource/Valparaíso,1998-3-11,"Cartagena,_Chile","http://dbpedia.org/resource/Cartagena,_Chile",male
2,Ralph_Barton,https://fr.wikipedia.org/wiki/Ralph_Barton,"['id=""Biographie"">Biographie[modifier', '|', '...",Ralph Waldo Emerson Barton (14 août 1891 – 19 ...,1891-8-14,"Kansas_City,_Missouri","http://dbpedia.org/resource/Kansas_City,_Missouri",1931-5-19,"New_York_City,_New_York","http://dbpedia.org/resource/New_York_City,_New...",male
8,Jack_Goldstein,https://fr.wikipedia.org/wiki/Jack_Goldstein,[],Jack Goldstein (* 27. September 1945 in Montre...,1945-9-27,Montreal,http://dbpedia.org/resource/Montreal,2003-3-14,"San_Bernardino,_California","http://dbpedia.org/resource/San_Bernardino,_Ca...",male
11,Paul_Markwitz,https://fr.wikipedia.org/wiki/Paul_Markwitz,"['id=""Biographie"">Biographie[modifier', '|', '...",Paul E. Markwitz (* 16. Juli 1908 in Berlin; †...,1908-7-16,Berlin,http://dbpedia.org/resource/Berlin,1968-03-19,Berlin,http://dbpedia.org/resource/Berlin,male
13,Virginia_Woolf,https://fr.wikipedia.org/wiki/Virginia_Woolf,"['id=""Biographie"">Biographie[modifier', '|', '...",Adeline Virginia Woolf (née Stephen; 25 Januar...,1882-01-25,Kensington,http://dbpedia.org/resource/Kensington,1941-03-28,Lewes,http://dbpedia.org/resource/Lewes,female


## Save to .csv 

In [24]:
data.to_csv("../data/Suicide_withDBPedia_Values_fr.csv",index=False, sep=";", encoding='utf-8-sig')

In [25]:
data_errors.to_csv("../data/Suicide_withDBPedia_Values_errors_fr.csv",index=False, sep=";", encoding='utf-8-sig')

## Match birthplace with countries

In [26]:
listeBirthplaceCountries = data["birthplace_clean"].values
countries = []
i = 0
len(listeBirthplaceCountries)

1118

In [27]:
for country in range(len(listeBirthplaceCountries)):
    if(country % 111 == 0):
        print((country/len(listeBirthplaceCountries)*100),"% du travail effectué !")
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?country
    WHERE { <http://dbpedia.org/resource/"""+listeBirthplaceCountries[country]+"""> dbo:country ?country}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if (results["results"]["bindings"] != []):
        results = results["results"]["bindings"][0]
        countries.append(results["country"]["value"])
    else:
        i+=1
        if (i%10 == 0):
            print("nombre de données manquantes :", i)
        countries.append("")

0.0 % du travail effectué !
nombre de données manquantes : 10
nombre de données manquantes : 20
nombre de données manquantes : 30
nombre de données manquantes : 40
9.928443649373882 % du travail effectué !
nombre de données manquantes : 50
nombre de données manquantes : 60
nombre de données manquantes : 70
nombre de données manquantes : 80
19.856887298747765 % du travail effectué !
nombre de données manquantes : 90
nombre de données manquantes : 100
nombre de données manquantes : 110
nombre de données manquantes : 120
29.785330948121647 % du travail effectué !
nombre de données manquantes : 130
nombre de données manquantes : 140
nombre de données manquantes : 150
39.71377459749553 % du travail effectué !
nombre de données manquantes : 160
nombre de données manquantes : 170
nombre de données manquantes : 180
49.642218246869405 % du travail effectué !
nombre de données manquantes : 190
nombre de données manquantes : 200
nombre de données manquantes : 210
nombre de données manquantes : 22

In [28]:
birthcountry_clean = [""]*len(countries)
for i in range(len(countries)):
    for j in range(28, len(countries[i])):
        birthcountry_clean[i] += countries[i][j]
data["birthCountry"] = birthcountry_clean
data.head()

,Name,Link,Content,abstract,birthdate,birthplace_clean,birthplace,deathdate,deathplace_clean,deathplace,gender,birthCountry
0,Adolfo_Couve,https://fr.wikipedia.org/wiki/Adolfo_Couve,"['id=""Biographie"">Biographie[modifier', '|', '...","Adolfo Couve Rioseco (Valparaíso, 28 de marzo ...",1940-3-28,Valparaíso,http://dbpedia.org/resource/Valparaíso,1998-3-11,"Cartagena,_Chile","http://dbpedia.org/resource/Cartagena,_Chile",male,Chile
2,Ralph_Barton,https://fr.wikipedia.org/wiki/Ralph_Barton,"['id=""Biographie"">Biographie[modifier', '|', '...",Ralph Waldo Emerson Barton (14 août 1891 – 19 ...,1891-8-14,"Kansas_City,_Missouri","http://dbpedia.org/resource/Kansas_City,_Missouri",1931-5-19,"New_York_City,_New_York","http://dbpedia.org/resource/New_York_City,_New...",male,United_States
8,Jack_Goldstein,https://fr.wikipedia.org/wiki/Jack_Goldstein,[],Jack Goldstein (* 27. September 1945 in Montre...,1945-9-27,Montreal,http://dbpedia.org/resource/Montreal,2003-3-14,"San_Bernardino,_California","http://dbpedia.org/resource/San_Bernardino,_Ca...",male,Canada
11,Paul_Markwitz,https://fr.wikipedia.org/wiki/Paul_Markwitz,"['id=""Biographie"">Biographie[modifier', '|', '...",Paul E. Markwitz (* 16. Juli 1908 in Berlin; †...,1908-7-16,Berlin,http://dbpedia.org/resource/Berlin,1968-03-19,Berlin,http://dbpedia.org/resource/Berlin,male,Germany
13,Virginia_Woolf,https://fr.wikipedia.org/wiki/Virginia_Woolf,"['id=""Biographie"">Biographie[modifier', '|', '...",Adeline Virginia Woolf (née Stephen; 25 Januar...,1882-01-25,Kensington,http://dbpedia.org/resource/Kensington,1941-03-28,Lewes,http://dbpedia.org/resource/Lewes,female,United_Kingdom


## Where the birthcountry requests failed (for analysis)

In [29]:
data[(data['birthCountry'] == "")]

,Name,Link,Content,abstract,birthdate,birthplace_clean,birthplace,deathdate,deathplace_clean,deathplace,gender,birthCountry
17,Jean-François_Adam,https://fr.wikipedia.org/wiki/Jean-François_Adam,"['id=""Biographie"">Biographie[modifier', '|', '...",Jean-François AdamFichier:Image manquante.jpg ...,1938-2-14,France,http://dbpedia.org/resource/France,1980-10-14,France,http://dbpedia.org/resource/France,male,
27,Pierre_Batcheff,https://fr.wikipedia.org/wiki/Pierre_Batcheff,"['id=""Biographie"">Biographie[modifier', '|', '...",Pierre Batcheff (born Piotr Bachev; 23 June 19...,1901-6-23,China,http://dbpedia.org/resource/China,1932-4-13,Paris,http://dbpedia.org/resource/Paris,male,
29,Rémy_Belvaux,https://fr.wikipedia.org/wiki/Rémy_Belvaux,"['id=""Biographie"">Biographie[modifier', '|', '...","Rémy Belvaux (*Namur, 1967 - † 4 de septiembre...",1966-11-10,Belgium,http://dbpedia.org/resource/Belgium,2006-9-4,Orry-la-Ville,http://dbpedia.org/resource/Orry-la-Ville,male,
32,Jonathan_Brandis,https://fr.wikipedia.org/wiki/Jonathan_Brandis,"['id=""Biographie"">Biographie[modifier', '|', '...","Jonathan Gregory Brandis (April 13, 1976 – Nov...",1976-4-13,"Danbury,_Connecticut","http://dbpedia.org/resource/Danbury,_Connecticut",2003-11-12,Los_Angeles,http://dbpedia.org/resource/Los_Angeles,male,
33,Claudio_Camaso,https://fr.wikipedia.org/wiki/Claudio_Camaso,"['id=""Biographie"">Biographie[modifier', '|', '...","Claudio Camaso, de son vrai nom Claudio Volont...",1934-1-12,Italy,http://dbpedia.org/resource/Italy,1977-9-16,Italy,http://dbpedia.org/resource/Italy,male,
34,Arthur_Edmund_Carewe,https://fr.wikipedia.org/wiki/Arthur_Edmund_Ca...,"['id=""Biographie"">Biographie[modifier', '|', '...",Arthur Edmund Carewe (parfois crédité Arthur C...,1884-12-30,Ottoman_Empire,http://dbpedia.org/resource/Ottoman_Empire,1937-04-22,"Santa_Monica,_California","http://dbpedia.org/resource/Santa_Monica,_Cali...",male,
35,Walmor_Chagas,https://fr.wikipedia.org/wiki/Walmor_Chagas,[],"Walmor Chagas, né le 28 août 1930 à Alegrete e...",1930-8-28,Brazil,http://dbpedia.org/resource/Brazil,2013-1-18,Guaratinguetá,http://dbpedia.org/resource/Guaratinguetá,male,
36,Leslie_Cheung,https://fr.wikipedia.org/wiki/Leslie_Cheung,"['id=""Biographie"">Biographie[modifier', '|', '...",(This is a Chinese name; the family name is Ch...,1956-9-12,Hong_Kong,http://dbpedia.org/resource/Hong_Kong,2003-4-1,"Central,_Hong_Kong","http://dbpedia.org/resource/Central,_Hong_Kong",male,
37,Ronny_Coutteure,https://fr.wikipedia.org/wiki/Ronny_Coutteure,"['id=""Biographie"">Biographie[modifier', '|', '...","Ronny Coutteure est un acteur, réalisateur et ...",1951-07-02,Belgium,http://dbpedia.org/resource/Belgium,2000-06-21,Fretin,http://dbpedia.org/resource/Fretin,male,
39,Karl_Dane,https://fr.wikipedia.org/wiki/Karl_Dane,"['id=""Biographie"">Biographie[modifier', '|', '...",25بك المحتوى هنا ينقصه الاستشهاد بمصادر. يرجى ...,1886-10-12,Denmark,http://dbpedia.org/resource/Denmark,1934-4-14,Los_Angeles,http://dbpedia.org/resource/Los_Angeles,male,


It could be due to the fact that birthplace_clean certains time does not contains the city, but the country of the birth. So it will be tries to find if the value is a country with a sparql request. 

if the value is a country, it has a capital. So a request is performed to know access to all of the values that are countries with a request that say "if the value has a capital, add it to the array"

In [30]:
capitals = ""
capital_clean = [""]*len(data)
for i in range(len(data)):
    if(data.iloc[i, -1] == ""):
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?capital
            WHERE { <http://dbpedia.org/resource/"""+data.iloc[i, 5]+"""> dbo:capital ?capital}
            """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        if(results["results"]["bindings"] != []):
            capital = results["results"]["bindings"][0]["capital"]["value"]
            for j in range(28, len(capital)):
                capital_clean[i] += capital[j]

## replace the value in birthCountry by birthplace value if the birthCountry is unknown and the birthplace have a capital (i.e., is a country)

In [31]:
for i in range(len(data)):
    if(birthcountry_clean[i] == "" and capital_clean[i] != ""):
        if(capital_clean[i] == "United_States"):
            capital_clean[i] = "United_States_of_America"
        print("capital :", capital_clean[i])
        birthcountry_clean[i] = birthplace_clean[i]
    elif(birthcountry_clean[i] =="List_of_historical_capitals_of_Hungary"):
        birthcountry_clean[i] = "Hungary"

capital : Paris
capital : Beijing
capital : City_of_Brussels
capital : Rome
capital : Bursa
capital : Brasília
capital : City_of_Brussels
capital : Copenhagen
capital : Washington,_D.C.
capital : Moscow
capital : Tokyo
capital : Berlin
capital : Washington,_D.C.
capital : Tokyo
capital : Seoul
capital : Rome
capital : Canberra
capital : Washington,_D.C.
capital : Budapest
capital : Buenos_Aires
capital : Paris
capital : Cairo
capital : Warsaw
capital : Budapest
capital : Paris
capital : Budapest
capital : Kiev
capital : Berlin
capital : Paris
capital : Berlin
capital : Nicosia
capital : Rome
capital : Saint_Helier,_Jersey
capital : Stockholm
capital : Washington,_D.C.
capital : Budapest
capital : Santiago
capital : Berlin
capital : Bursa
capital : Helsinki
capital : Vitebsk
capital : Paris
capital : Berlin
capital : Havana
capital : Tokyo
capital : Budapest
capital : Bogotá
capital : Paris
capital : Sofia
capital : Vilnius
capital : Prague
capital : Beijing
capital : Tehran
capital : T

## changing the value of the column in the dataframe and save it into a .csv file

In [37]:
data["birthcountry"] = birthcountry_clean

In [33]:
data.to_csv("../data/Suicide_withDBPedia_Values_fr.csv",index=False, sep=";", encoding='utf-8-sig')

In [ ]:
for name in range(len(listeName)):
    if(name % 201 == 0):
        print((name/len(listeName)*100),"% du travail effectué !")
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?abstract ?birthdate ?deathdate ?birthplace ?deathplace ?gender
    WHERE { <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:abstract ?abstract.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:birthDate ?birthdate.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathDate ?deathdate.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:birthPlace ?birthplace.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathPlace ?deathplace.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> foaf:gender ?gender}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if (results["results"]["bindings"] != []):
        results = results["results"]["bindings"][0]
        abstract.append(results["abstract"]["value"])
        birthdate.append(results["birthdate"]["value"])
        birthplace.append(results["birthplace"]["value"])
        deathdate.append(results["deathdate"]["value"])
        deathplace.append(results["deathplace"]["value"])
        gender.append(results["gender"]["value"])
    else:
        i+=1
        if (i%10 == 0):
            print("nombre de données manquantes :", i)
        abstract.append("")
        birthdate.append("")
        birthplace.append("")
        deathdate.append("")
        deathplace.append("")
        gender.append("")
print("il manque", (i/len(listeName))*100,"% des données")

In [77]:
deathdate = []
deathplace = []
i=0
fails=[]

In [78]:
for name in range(len(listeName)):
    if(name % 201 == 0):
        print((name/len(listeName)*100),"% du travail effectué !")
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?deathdate ?deathplace
    WHERE {
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathDate ?deathdate.
    <http://dbpedia.org/resource/"""+listeName[name]+"""> dbo:deathPlace ?deathplace.}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    if (results["results"]["bindings"] != []):
        results = results["results"]["bindings"][0]
        deathdate.append(results["deathdate"]["value"])
        deathplace.append(results["deathplace"]["value"])
    else:
        sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
        sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?deathdate ?deathplace
        WHERE {
        <http://fr.dbpedia.org/resource/"""+listeName[name]+"""> dbpedia-owl:deathDate ?deathdate.
        <http://fr.dbpedia.org/resource/"""+listeName[name]+"""> dbpedia-owl:deathPlace ?deathplace.}
        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        if (results["results"]["bindings"] != []):
            results = results["results"]["bindings"][0]
            deathdate.append(results["deathdate"]["value"])
            deathplace.append(results["deathplace"]["value"])
        else:
            fails.append([listeName[name],results])
            i+=1
            if (i%10 == 0):
                print("nombre de données manquantes :", i)
                deathdate.append("")
                deathplace.append("")
print("il manque", (i/len(listeName))*100,"% des données")

0.0 % du travail effectué !
nombre de données manquantes : 10
nombre de données manquantes : 20
nombre de données manquantes : 30
nombre de données manquantes : 40
9.575988565983803 % du travail effectué !
nombre de données manquantes : 50
nombre de données manquantes : 60
nombre de données manquantes : 70
nombre de données manquantes : 80
nombre de données manquantes : 90
19.151977131967605 % du travail effectué !
nombre de données manquantes : 100
nombre de données manquantes : 110
nombre de données manquantes : 120
nombre de données manquantes : 130
nombre de données manquantes : 140
nombre de données manquantes : 150
28.727965697951408 % du travail effectué !
nombre de données manquantes : 160
nombre de données manquantes : 170
nombre de données manquantes : 180
nombre de données manquantes : 190
nombre de données manquantes : 200
nombre de données manquantes : 210
nombre de données manquantes : 220
nombre de données manquantes : 230
38.30395426393521 % du travail effectué !
nombre

In [80]:
fails

[['Boaz_Arad',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['Mark_Campos',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['Michel_Caron_(artiste)',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['Cassandre_(graphiste)',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['Alberto_Della_Valle',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['Serge_Legat',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace']},
   'results': {'distinct': False, 'ordered': True, 'bindings': []}}],
 ['François_Lunven',
  {'head': {'link': [], 'vars': ['deathdate', 'deathplace'